In [1]:
import socket, threading, time, random
from datetime import datetime 

class Rover:
    def __init__(self):
        self.s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Command Center Socket 
        self.s.bind(('127.0.0.1', 8890))  
        self.rover_addr = ('127.0.0.1', 8891)    
        self.flag = threading.Event() # block the main thread before receiving response
        self.token = b''
        self.receive_thread = threading.Thread(target=self.receiver)
        self.receive_thread.daemon = True
        self.receive_thread.start() 
        
    def receiver(self): 
        while True:
            data = b'' 
            try: 
                data, addr = self.s.recvfrom(1024)  
                if data == self.token + b'**RT**': # RT = Roger That
                    print(data)  
                    self.flag.set() 
            except socket.error as exc:
                self.log.append( (datetime.now().isoformat(), f"Caught exception socket.error : {exc}") )   
         
    def send_command(self, command, timeout=5):  
        self.flag.clear()
        self.token = str(random.randint(1, 65535)).zfill(5).encode('utf-8')
        self.s.sendto(command.encode('utf-8')+self.token+b'**RT**', self.rover_addr)
        if not self.flag.wait(timeout):
            print(f'No response from Rover in {timeout} seconds, Check if it is offline!')

rover = Rover()

In [2]:
rover.s.sendto(b'Just send a UDP packet ~ **RT**', ('127.0.0.1', 8891) )

31

In [3]:
rover.send_command('Check if the rover responds')

b'03050**RT**'


### At this point, kill the server to see the difference~

In [4]:
rover.send_command('What if the rover is offline', timeout=5) ## Main block will be blocked 5s if get no responded 

No response from Rover in 5 seconds, Check if it is offline!
